In [1]:
!pip install -q spacy scikit-learn sentence-transformers
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 97.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload d

In [9]:
import pandas as pd
import numpy as np
import spacy
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv('/content/UpdatedResumeDataSet.csv')  # Update path in Colab
df = df[['Category', 'Resume']]  # Ensure relevant columns

Text Preprocessing (spaCy)

In [5]:
nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(tokens)

df['Cleaned_Resume'] = df['Resume'].apply(clean_text)


In [5]:
df.head(10)

,Category,Resume,Cleaned_Resume
0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail uit rgpv data scientist data ...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...
5,Data Science,"SKILLS C Basics, IOT, Python, MATLAB, Data Sci...",skill c basic iot python matlab datum science ...
6,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau datum visualization r stu...
7,Data Science,Education Details \r\n B.Tech Rayat and Bahr...,education detail rayat bahra institute enginee...
8,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skill ability quickly grasp technical...
9,Data Science,Expertise â Data and Quantitative Analysis â...,expertise datum quantitative analysis decision...


BERT + Cosine

In [ ]:
# -- BERT Embeddings
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
resume_embeddings = bert_model.encode(df['Resume'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
job_description = "We are hiring a data scientist with experience in machine learning, Python, and deep learning frameworks like TensorFlow or PyTorch."
job_embedding = bert_model.encode([job_description])


Cosine Similarity

In [ ]:
similarities = cosine_similarity(job_embedding, resume_embeddings).flatten()
df['Similarity'] = similarities

# Sort by similarity descending
ranked_resumes = df.sort_values(by='Similarity', ascending=False).reset_index(drop=True)

# Display Top N Candidates
top_n = 10
print(f"\nTop {top_n} Candidates Matching the Job Description:\n")
for idx, row in ranked_resumes.head(top_n).iterrows():
    print(f"Rank {idx+1}:")
    print(f"Category     : {row['Category']}")
    print(f"Match Score  : {round(row['Similarity'], 4)}")
    print(f"Resume Snippet: {row['Resume'][:300]}...")
    print("="*80)


Top 10 Candidates Matching the Job Description:

Rank 1:
Category     : Data Science
Match Score  : 0.5867
Resume Snippet: Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language pr...
Rank 2:
Category     : Data Science
Match Score  : 0.5867
Resume Snippet: Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language pr...
Rank 3:
Category     : Data Science
Match Score  : 0.5867
Resume Snippet: Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. *

From above output we can observe that BERT gives better accuracy than TFIDF

In [ ]:
import pickle

# Save the resume embeddings to disk
with open('final_resume_embeddings.pkl', 'wb') as f:
    pickle.dump(resume_embeddings, f)


In [ ]:
df.to_csv("cleaned_resumes_with_similarity.csv", index=False)


TF-IDF + Cosine Similarity

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Clean resumes using spaCy
# df['Cleaned_Resume'] = df['Resume'].apply(clean_text)

# Fit TF-IDF vectorizer on resumes
tfidf = TfidfVectorizer(max_features=1000)
resume_tfidf = tfidf.fit_transform(df['Cleaned_Resume'])

# Define and clean job description
job_description = "We are hiring a data scientist with experience in machine learning, Python, and deep learning frameworks like TensorFlow or PyTorch."
job_desc_cleaned = clean_text(job_description)

# Transform job description into TF-IDF vector
job_tfidf = tfidf.transform([job_desc_cleaned])

# Compute cosine similarity between job description and resumes
similarities_tfidf = cosine_similarity(job_tfidf, resume_tfidf).flatten()
df['Similarity_TFIDF'] = similarities_tfidf

# Rank resumes by similarity
ranked_tfidf = df.sort_values(by='Similarity_TFIDF', ascending=False).reset_index(drop=True)

# Display Top 5 TF-IDF matches
print("\nTop 5 Candidates (TF-IDF Matching):\n")
for i, row in ranked_tfidf.head(10).iterrows():
    print(f"Rank {i+1}:")
    print(f"Category     : {row['Category']}")
    print(f"Match Score  : {round(row['Similarity_TFIDF'], 4)}")
    print(f"Resume Snippet: {row['Resume'][:300]}...")
    print("=" * 80)



Top 5 Candidates (TF-IDF Matching):

Rank 1:
Category     : Data Science
Match Score  : 0.6048
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprience - Less than 1 year months
Machine Learning- Exprience - Less than 1 year months
Tensorflow- Exprience - Less than 1 year months
Scikit- Exprien...
Rank 2:
Category     : Data Science
Match Score  : 0.6048
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprience - Less than 1 year months
Machine Learning- Exprience - Less than 1 year months
Tensorflow- Exprience - Less than 1 year months
Scikit- Exprien...
Rank 3:
Category     : Data Science
Match Score  : 0.6048
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprience - 

In [12]:

# Save the trained TF-IDF vectorizer
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


Word2Vec / GloVe + Average Embedding + Cosine Similarity

In [12]:
!pip install -q gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [8]:
# !pip install numpy==1.23.5 gensim --upgrade --force-reinstall


In [ ]:

import gensim.downloader as api

# Load pre-trained word embeddings (e.g., GloVe)
w2v_model = api.load("glove-wiki-gigaword-100")  # Or try word2vec-google-news-300


In [7]:
def get_avg_embedding(text):
    words = clean_text(text).split()
    valid_words = [w for w in words if w in w2v_model]
    if not valid_words:
        return np.zeros(100)
    return np.mean([w2v_model[w] for w in valid_words], axis=0)

resume_avg_vecs = np.array([get_avg_embedding(r) for r in df['Resume']])

# Step 4: Job Description
job_description = "We are hiring a data scientist with experience in machine learning, Python, and deep learning frameworks like TensorFlow or PyTorch."
job_vec_avg = get_avg_embedding(job_description).reshape(1, -1)

# Step 5: Cosine Similarity & Ranking
similarities_avg = cosine_similarity(job_vec_avg, resume_avg_vecs).flatten()
df['Similarity_AvgW2V'] = similarities_avg

ranked_avg = df.sort_values(by='Similarity_AvgW2V', ascending=False).reset_index(drop=True)

# Step 6: Display Top 10 Matches
print("\nTop 10 Candidates (Avg GloVe Embedding):\n")
for i, row in ranked_avg.head(10).iterrows():
    print(f"Rank {i+1}:")
    print(f"Category     : {row['Category']}")
    print(f"Match Score  : {round(row['Similarity_AvgW2V'], 4)}")
    print(f"Resume Snippet: {row['Resume'][:300]}...")
    print("=" * 80)


Top 10 Candidates (Avg GloVe Embedding):

Rank 1:
Category     : Data Science
Match Score  : 0.9512
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprience - Less than 1 year months
Machine Learning- Exprience - Less than 1 year months
Tensorflow- Exprience - Less than 1 year months
Scikit- Exprien...
Rank 2:
Category     : Data Science
Match Score  : 0.9512
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprience - Less than 1 year months
Machine Learning- Exprience - Less than 1 year months
Tensorflow- Exprience - Less than 1 year months
Scikit- Exprien...
Rank 3:
Category     : Data Science
Match Score  : 0.9512
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprien

In [13]:

# Save the word embedding model (Word2Vec or GloVe)
w2v_model.save("w2v_model.model")  # Works if it's a gensim model

# Save the resume vectors (numpy array)
np.save("resume_avg_embeddings.npy", resume_avg_vecs)



1. Accept PDF resumes from frontend
Your frontend should send user-uploaded PDF files (via form/file upload) to your backend API (e.g., Flask, FastAPI, Node.js).

2. Extract text from PDF resumes
On the backend, before matching, you must extract the raw text from PDFs using a library such as:

# For Python backend
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text
    
Alternatives:

pdfplumber – better layout handling

pdfminer.six – more control over structured extraction

3. Clean and preprocess the extracted text
You must use the same clean_text() function that you used during training (with spaCy tokenization and lemmatization). This ensures consistent feature representation.

python

resume_text = extract_text_from_pdf("user_resume.pdf")
cleaned_resume = clean_text(resume_text)

4. Convert to embedding or vector
Depending on your model:

TF-IDF:

python
Copy
Edit
resume_vector = tfidf_vectorizer.transform([cleaned_resume])
BERT:

python
Copy
Edit
resume_vector = bert_model.encode([resume_text])
GloVe (Avg Embeddings):

python
Copy
Edit
resume_vector = get_avg_embedding(resume_text).reshape(1, -1)

5. Match with Job Description
Preprocess the job description the same way

Compute cosine similarity

Return top-N match score or ranking

python
Copy
Edit
score = cosine_similarity(job_vector, resume_vector).flatten()[0]


In [ ]:
# For Python backend
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

resume_text = extract_text_from_pdf("user_resume.pdf")
cleaned_resume = clean_text(resume_text)

resume_vector = tfidf_vectorizer.transform([cleaned_resume])

resume_vector = bert_model.encode([resume_text])

resume_vector = get_avg_embedding(resume_text).reshape(1, -1)


[Frontend PDF Upload] → [Backend]

 → Extract Text (PyPDF2/pdfplumber)

 → Clean + Preprocess

 → Convert to Feature Vector (TF-IDF/BERT)

 → Compare with Job Description

 → Return Similarity Score / Rank
